In [1]:
number_of_frames_to_record=10000 #3000
background_chunk=100
frame_width=640
frame_height=480
fps=30
preset_json_file="d415paramset_640480.json"
import gzip
import shutil

In [2]:
import sys
sys.path.append('/usr/local/lib')

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib.colors as mcol
import matplotlib.cm as cm
import math
import imageio
import pickle
import gc
import os
import time
import json
import pyrealsense2 as rs
import gzip
import cv2
from scipy import stats
from scipy import ndimage #for finding com
from scipy.misc import imsave
from itertools import cycle
cycol = cycle('bgrcmk') #here I'm generating a list of colors I can plot

In [4]:
# Enabling Advanced Mode

DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

try:
    dev = find_device_that_supports_advanced_mode()
    advnc_mode = rs.rs400_advanced_mode(dev)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Loop until we successfully enable advanced mode
    while not advnc_mode.is_enabled():
        print("Trying to enable advanced mode...")
        advnc_mode.toggle_advanced_mode(True)
        # At this point the device will disconnect and re-connect.
        print("Sleeping for 5 seconds...")
        time.sleep(5)
        # The 'dev' object will become invalid and we need to initialize it again
        dev = find_device_that_supports_advanced_mode()
        advnc_mode = rs.rs400_advanced_mode(dev)
        print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    
except Exception as e:
    print(e)
pass

Found device that supports advanced mode: Intel RealSense D415
Advanced mode is enabled


In [5]:
# Loading presaved parameters 
  
with open(preset_json_file) as json_data: #Json file to be loaded #NOTE im using default NOT the json that I just created above
    starting_params = json.load(json_data)
    starting_params=str(starting_params).replace("'", '\"') #IF YOU DON't DO THIS IT WILL NOT WORK with the librealsense sdk
    print(starting_params)

{"aux-param-autoexposure-setpoint": "400", "aux-param-colorcorrection1": "0.461914", "aux-param-colorcorrection10": "-0.553711", "aux-param-colorcorrection11": "-0.553711", "aux-param-colorcorrection12": "0.0458984", "aux-param-colorcorrection2": "0.540039", "aux-param-colorcorrection3": "0.540039", "aux-param-colorcorrection4": "0.208008", "aux-param-colorcorrection5": "-0.332031", "aux-param-colorcorrection6": "-0.212891", "aux-param-colorcorrection7": "-0.212891", "aux-param-colorcorrection8": "0.68457", "aux-param-colorcorrection9": "0.930664", "aux-param-depthclampmax": "65536", "aux-param-depthclampmin": "0", "aux-param-disparityshift": "0", "controls-autoexposure-auto": "True", "controls-autoexposure-manual": "33000", "controls-color-autoexposure-auto": "True", "controls-color-autoexposure-manual": "166", "controls-color-backlight-compensation": "0", "controls-color-brightness": "0", "controls-color-contrast": "50", "controls-color-gain": "64", "controls-color-gamma": "300", "co

In [6]:
#Experimental Loop

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, fps)
config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, 30)
advnc_mode.load_json(starting_params) #this is where I start with the Defualt Json
# Start streaming
cfg = pipeline.start(config)
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()

#If I was applying a spatial filter...
#sf=rs.spatial_filter() 
#sf.set_option()

tick=0; 

In [ ]:
start_of_loop=time.time()
with open('pythondepthdata.bin','ab') as depthbinarydata, open('pythoncolordata.bin', 'ab') as colorbinarydata: #While the file called "pythonedpthdata.bin" is open for appending binary, 
    try:
        while tick<number_of_frames_to_record: 
            tick=tick+1 
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            #depth_frame = sf.process(depth_frame); #apply a filter like this

            color_frame = frames.get_color_frame()

           
            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            depth_image.tofile(depthbinarydata) #save data as binarydata (specified by the with open as line)
            
            color_image = np.asanyarray(color_frame.get_data())
            color_image.tofile(colorbinarydata) #save data as binarydata (specified by the with open as line)

            if tick==background_chunk:
                output=input("please insert object to record and enter YYMMDD_exp######  ")
        
            with open('depth_ts.txt', 'a') as ts:
                ts.write('%s\n' %rs.frame.get_timestamp(frames)) #write time stamp float as a string to a new line

    finally:

        # Stop streaming
        pipeline.stop()
        end_of_loop=time.time() #end timer
        print("Time Elapsed = ", end_of_loop - start_of_loop)
        serialized_string = advnc_mode.serialize_json() #getting the current parameters as a serialized string
        as_json_object = json.loads(serialized_string) #loading the json object from the serialized string
        json_used_in_experiment = str(as_json_object).replace("'", '\"') #converting object into string that can be printed
        print("Final Parameters = ", json_used_in_experiment)

please insert object to record and enter YYMMDD_exp######  180730_000001


In [ ]:
depthfilename='%s_raw_depth.gz'%output
colorfilename='%s_raw_color.gz'%output
timestampfilename="%s_depth_ts.txt"%output

In [ ]:
#compress color data
file=open('pythoncolordata.bin','rb')
filedata = file.read()
file.close()

! rm 'pythoncolordata.bin'

outF = gzip.GzipFile(colorfilename, 'wb')
outF.write(filedata)
outF.close()

In [ ]:
#compress depth data
file=open('pythondepthdata.bin','rb')
filedata = file.read()
file.close()

! rm 'pythondepthdata.bin'

outF = gzip.GzipFile(depthfilename, 'wb')
outF.write(filedata)
outF.close()

In [ ]:
os.rename('depth_ts.txt', timestampfilename)

In [ ]:
with open(output+'_variables', 'wb') as variables:
    pickle.dump([number_of_frames_to_record,frame_height,frame_width,fps,background_chunk],variables)

In [ ]:
exp_numb=0
frame_width=640
frame_height=480
fps=90
number_of_frames_to_record=50000 #3000
background_chunk=100 #I want to use 100 frames for recording the background
preset_json_file="d415paramset_640480.json"
grand_frame=int(exp_numb*(number_of_frames_to_record-background_chunk))
min_range=0
max_range=800 #1000
data_chunk_size=3000 #1500
box_size=192 #this is how big I want my video to be (200 by 200) #YOU WILL GET ERRORS IF YOU'RE BOX IS BIGGER THAN THE MIDDLE OF THE IMAGE (x or y)
dpi=80
stringency=10 #10

In [ ]:
#If only testing then remove data
#! rm 'depth_ts.txt'

In [ ]:
!rm depthdata.bin

In [ ]:
# if data looks good then scp to cluster for processing 
# scp 'pythondepthdata.bin' ra206@transfer.rc.hms.harvard.edu:~
# scp 'pythoncolordata.bin' ra206@transfer.rc.hms.harvard.edu:~
# scp 'depth_ts.txt' ra206@transfer.rc.hms.harvard.edu:~

In [ ]:
# while this code works the data gets to enormous sizes. Need to be able to compress on the fly